In [8]:
# v0.2 is extracting Age with 99+% accuracy.
# v0.3 is extracting Age and Gender with 99+% accuracy.
# v0.4 is for Name

In [9]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt                                               #importing libraries
import matplotlib as mpl
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pdf2image
import re
from PIL import Image
from pytesseract import pytesseract
import numpy


mpl.rcParams['image.cmap'] = 'gray'

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"    #providing path of tesseract
pytesseract.tesseract_cmd = path_to_tesseract


In [10]:
def convert_pdf_to_image(document, dpi):
    images = []
    images.extend(
                    list(
                        map(
                            lambda image: cv2.cvtColor(
                                np.asarray(image), code=cv2.COLOR_RGB2BGR             
                            ),                                                     #converting all pages of pdf into images and storing them in a list
                            pdf2image.convert_from_path(document, dpi=dpi, poppler_path=r"C:\Users\Amandeep Singh\Downloads\Release-23.05.0-0\poppler-23.05.0\Library\bin"),
                        )
                    )
                )
    return images

In [11]:
def f(x):
    
    global horizontal_lines_img, verticle_lines_img, kernel
    kernel_length = np.array(img).shape[1]//x

    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)                    #Creating Horizontal&Vertical Lines images
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    cv2.imwrite("Images/verticle_lines.jpg",verticle_lines_img)
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
    


In [12]:
def sort_contours(cnts, method):
                                                                      #Sorting Contours
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),key=lambda b:[b[1][1],b[1][0]] , reverse=False))

    return (cnts, boundingBoxes)


In [13]:
start_pg = 81                       #Providing Starting and End files range
end_pg = 90

In [ ]:
for pg in range(start_pg,end_pg+1):
    
    images = convert_pdf_to_image(str(pg)+'.pdf', 300)
    
    final_lis = []
    for i in range(2,len(images)-1):
        img = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)                      #Converting color space of images to GrayScale

        (thresh, img_bin) = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY|     cv2.THRESH_OTSU)    #performing thresholding

        img_bin = 255-img_bin 
        cv2.imwrite("Images/Image_bin.jpg",img_bin)

        interact(f,  x=widgets.IntSlider(min=1,max=255,step=1,value=50));

        alpha = 0.5                                              
        beta = 1.0 - alpha                                                 #blending of horizontal&vertical lines images
        img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
        img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
        (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)    #contour detection
        (contours, boundingBoxes) = sort_contours(contours,"top-to-bottom")          #sorting contours

        g = []                                                 #taking only required rectangular boxes
        for c in contours:
            x, y, w, h = cv2.boundingRect(c)
            if (w<800 and w > 700 and h>320):
                g.append([x,y,w,h])
        
        a = []
        for i in range(len(g)):
            if i == 0:
                a.append(g[i])
            else:
                if np.abs(g[i][0] - g[i-1][0]) < 50:
                    pass
                else:
                    a.append(g[i])

        for i in a:
            new_img = img[i[1]:i[1]+i[3], i[0]:i[0]+i[2]]
            dic = {}
            
            #OCR getting all details (regular)
            text = pytesseract.image_to_string(new_img,config='--psm 6')  #extracting text from image
            
            #OCR getting name, relation_name, and gender (extracting text and single spaces only)
            text_name = pytesseract.image_to_string(new_img, config = r'--psm 6 --oem 3 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "')
            text_for_name_relation_gender = text_name.split()
            
            #OCR getting age only (extracting numbers only)
            text_age = pytesseract.image_to_string(new_img, config='--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789')
            temp_text_age = text_age.split()
            text_after_config = temp_text_age[-1].strip()
            
            #OCR getting House no only
            text_house = pytesseract.image_to_string(new_img, config = r'--psm 6 --oem 1 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789/-.,"')
            text_house = text_house.split()
            
            #================
            #POST PROCESSING
            #================
            
            #Finding Name [OK]
            temp_list_for_name = []
            found_name = False
            for item in text_for_name_relation_gender:                        
                if item == "Name" or item == "Nama" or item == "Narne":
                    found_name = True
                    continue

                if found_name and (item == "Father" or item == "Mother" or item == "Husband" or item == "Other" or item == "Others" or item == "Husdand"):
                    break

                if found_name:
                    temp_list_for_name.append(item)

            Name = " ".join(temp_list_for_name)
            dic["Name"] = Name       
            
            #Finding Age [OK]
            match6 = re.search(r"\nAge(.*):(.*)Gender",text)
            text_before_config = ""
            if match6:
                text_before_config = text[match6.start()+1:match6.end()-6].split(":")[1]
                text_before_config = text_before_config.strip()
                            
            if text_before_config.isdigit() and len(text_before_config) <= 2:
                dic["Age"] = text_before_config
            else:
                if text_after_config.isdigit():
                    if len(text_after_config) <= 2:
                        dic["Age"] = text_after_config
                    else:
                        temp_age = text_after_config
                        temp_age = list(temp_age)
                        final_age = temp_age[-2]+temp_age[-1]
                        dic["Age"] = final_age
                        
            #Finding Gender [OK]
            temp_list_for_gender = []
            for element in text_for_name_relation_gender:
                if element == "Age":
                    index = text_for_name_relation_gender.index(element)
                    temp_list_for_gender = text_for_name_relation_gender[index:]
                    
                    for ele in temp_list_for_gender:
                        if isinstance(ele, str) and (ele.startswith("M")):
                            dic["Gender"] = "Male"
                            break
                        elif isinstance(ele, str) and (ele.startswith("F")):
                            dic["Gender"] = "Female"
                            break
                        elif isinstance(ele, str) and (ele.startswith("T")):
                            dic["Gender"] = "Third Gender"
                            break
                        else:
                            dic["Gender"] = ""                    
                    
#                     if temp_list_for_gender[2].startswith("M"):
#                         dic["Gender"] = "Male"
#                     elif temp_list_for_gender[2].startswith("F"):
#                         dic["Gender"] = "Female"
#                     elif temp_list_for_gender[2].startswith("T"):
#                         dic["Gender"] = "Third Gender"
#                     elif temp_list_for_gender[3].startswith("M"):
#                         dic["Gender"] = "Male"
#                     elif temp_list_for_gender[3].startswith("F"):
#                         dic["Gender"] = "Female"
#                     elif temp_list_for_gender[3].startswith("T"):
#                         dic["Gender"] = "Third Gender"
#                     else:
#                         dic["Gender"] = ""

            #Finding father/mother/husband name [OK]
            found_element = None
            start_index = None
            temp_list = []

            for i in range(len(text_for_name_relation_gender)):
                if text_for_name_relation_gender[i] == "Father":
                    found_element = "Father"
                    start_index = i + 2
                    break
                elif text_for_name_relation_gender[i] == "Mother":
                    found_element = "Mother"
                    start_index = i + 2
                    break
                elif text_for_name_relation_gender[i] == "Husband" or text[i] == "Husdand":
                    found_element = "Husband"
                    start_index = i + 2
                    break

            if found_element is not None:
                for j in range(start_index, len(text_for_name_relation_gender)):
                    if text_for_name_relation_gender[j] == "Photo" or text_for_name_relation_gender[j] == "House":
                        break
                    temp_list.append(text_for_name_relation_gender[j])

            relation_name = " ".join(temp_list)
            
            if found_element == "Father":
                dic["FatherName"] = relation_name
            if found_element == "Mother":
                dic["MotherName"] = relation_name
            if found_element == "Husband":
                dic["HusbandName"] = relation_name
            
            #Finding House no
            temp_list_for_house_no = []
            house_no = None
            final_house_no = None

            starting_index = 0
            ending_index = 0
            
            for n in text_house:
                if n.startswith("House"):
                    starting_index = text_house.index(n)
                
                if n.startswith("Photo") or n.startswith("Phato") or n.startswith("Age"):
                    ending_index = text_house.index(n)
                    
            temp_list_for_house_no = text_house[starting_index:ending_index]

            house_no = " ".join(temp_list_for_house_no)

            patterns = r"HouseNumbers|HouseNumbert|HouseNumber|HouseNurnber|HouseNumbber|HouseNurmnber|HouseNurmber|HouseNurber|HouseNumnber|Photo|Phato|Available|Availal|Pho|Avail|i|Py|Avai|Availab|"
            final_house_no = re.sub(patterns, "", house_no)
            final_house_no = re.sub(patterns, "", house_no)
            final_house_no = final_house_no.strip()
            final_house_no = " " + final_house_no

            dic["HouseNumber"] = final_house_no
            
#             match8 = re.search(r"BXW(.*)|TB(.*)",text)
#             if match8:
#                 dic['Voter_id'] = text[match8.start():match8.end()]

            final_lis.append(dic)
            
    sd = pd.DataFrame(final_lis)
    sd = sd.astype({"HouseNumber": str})

    sd['Party Inclination'] = ''
    sd['Caste'] = ''

    sd[['Name','Age','Gender',"FatherName","HusbandName","MotherName","HouseNumber","Party Inclination","Caste"]].to_csv(str(pg) + ".csv")

    print(pg, "completed")

    
    

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

In [7]:
# sd

In [8]:
# sd['Name'].isnull().sum()

In [9]:
# sd['Gender'].isnull().sum()

In [10]:
# sd.to_excel('sample.xlsx')